### kakao 주소를 위도 경도로 변환 
- GET 방식 주소  
- https://dapi.kakao.com/v2/local/search/address.json
- 쿼리 파라미터 : query=주소

- 헤더 : Authorization: KakaoAK ${REST_API_KEY}
  

In [17]:
import requests, json
address = '서울 마포구 양화로 122 1층'
url = 'https://dapi.kakao.com/v2/local/search/address.json?query='+address
print(f'url:{url}')
#헤더
header = {'Authorization':'KakaoAK f30246ef4b6ee8da3e6e4c468782192c'} 

address_json=json.loads(   str(requests.get(url, headers=header).text))
print(f'address_json:{address_json}')
#print(address_json['documents'][0]['address'])
address_name=address_json['documents'][0]['address']

lat_lng={"lat":str(address_name['y']),'lng':str(address_name['x'])}
print(f'{address_name['address_name']},{lat_lng}')



url:https://dapi.kakao.com/v2/local/search/address.json?query=서울 마포구 양화로 122 1층
address_json:{'documents': [{'address': {'address_name': '서울 마포구 서교동 354-8', 'b_code': '1144012000', 'h_code': '1144066000', 'main_address_no': '354', 'mountain_yn': 'N', 'region_1depth_name': '서울', 'region_2depth_name': '마포구', 'region_3depth_h_name': '서교동', 'region_3depth_name': '서교동', 'sub_address_no': '8', 'x': '126.920603903509', 'y': '37.5540720468966'}, 'address_name': '서울 마포구 양화로 122', 'address_type': 'ROAD_ADDR', 'road_address': {'address_name': '서울 마포구 양화로 122', 'building_name': 'LAB7', 'main_building_no': '122', 'region_1depth_name': '서울', 'region_2depth_name': '마포구', 'region_3depth_name': '서교동', 'road_name': '양화로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '126.920603903509', 'y': '37.5540720468966', 'zone_no': '04038'}, 'x': '126.920603903509', 'y': '37.5540720468966'}], 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}
서울 마포구 서교동 354-8,{'lat': '37.5540720468966', 'lng': 

### lunch_20250828.csv 파일(주소)을 읽어 위도 경도 구하기
- 주소로 위도 경도 찾기
- lunch_20250828.csv 파일 읽기

In [18]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [35]:
import pandas as pd
import requests, json

df = pd.read_csv('lunch_20250828.csv', encoding='CP949')
df.head()

#df.info()
print(df.index[:]) #RangeIndex(start=0, stop=4, step=1)

#print(df.loc[0,'addr'])
#print(df.loc[1,'addr'])
#print(df.loc[2,'addr'])
#print(df.loc[3,'addr'])

#주소 -> 위도, 경도
url = 'https://dapi.kakao.com/v2/local/search/address.json?query='
#헤더
header = {'Authorization':'KakaoAK f30246ef4b6ee8da3e6e4c468782192c'} 


for i in df.index[:]:
    #print(f'i:{i}')
    #print(df.loc[i,'addr'])
    address = df.loc[i,'addr']
    addUrl = url + address
    #print(f'addUrl:{addUrl}')
    address_json=json.loads(   str(requests.get(addUrl, headers=header).text))
    address=address_json['documents'][0]['address']

    lat_lng={"lat":str(address['y']),"lng":str(address['x'])}

    name = df.loc[i,'name']
    print(f'{name},{address['address_name']},lat_lng:{lat_lng}')
    

    





RangeIndex(start=0, stop=4, step=1)
진진,서울 마포구 서교동 375-35,lat_lng:{'lat': '37.553806747709', 'lng': '126.918422364162'}
일식,서울 마포구 노고산동 56-45,lat_lng:{'lat': '37.5549520156765', 'lng': '126.932732694506'}
베이글랜드 홍대점,서울 마포구 서교동 332-15,lat_lng:{'lat': '37.5554919160248', 'lng': '126.925993298868'}
요거프레소,서울 마포구 서교동 354-8,lat_lng:{'lat': '37.5540720468966', 'lng': '126.920603903509'}


### CSV 파일을 읽어 홍대 맛집 지도 생성
- lunch_20250828.csv
- 중앙: 위도, 경도(37.5540720468966, 126.920603903509)
```python
folium.Map(
    location: Optional[Sequence[float]] = None,
    width: Union[str, float] = '100%',
    height: Union[str, float] = '100%',
    left: Union[str, float] = '0%',
    top: Union[str, float] = '0%',
    position: str = 'relative',
    tiles: Union[str, folium.raster_layers.TileLayer, NoneType] = 'OpenStreetMap',
    attr: Optional[str] = None,
    min_zoom: Optional[int] = None,
    max_zoom: Optional[int] = None,
    zoom_start: int = 10,
    min_lat: float = -90,
    max_lat: float = 90,
    min_lon: float = -180,
    max_lon: float = 180,
    max_bounds: bool = False,
    crs: str = 'EPSG3857',
    control_scale: bool = False,
    prefer_canvas: bool = False,
    no_touch: bool = False,
    disable_3d: bool = False,
    png_enabled: bool = False,
    zoom_control: Union[bool, str] = True,
    font_size: str = '1rem',
    **kwargs: Union[str, float, bool, Sequence, dict, NoneType],
)
```

In [45]:
import pandas as pd
import folium

#csv 파일 읽기
df = pd.read_csv('lunch_20250828.csv', encoding='CP949')
df.head()

#지도 중앙
center = [37.5540720468966, 126.920603903509]

map=folium.Map(location=center, zoom_start=15)
#print(df.loc[0,['latitude','logtitude']])

print(df.loc[i,['name','cate']])
for i in df.index[:]:
    folium.Circle(
         location=df.loc[i,['latitude','longitude']],
         tooltip = df.loc[i,['name','cate']], 
         radius=10,
        color ='blue'
     ).add_to(map)    


map.save('hong.html')
map

name    요거프레소
cate     cafe
Name: 3, dtype: object


C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\folium\utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\folium\utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  coords = (location[0], location[1])
C:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\folium\utilities.py:95: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame be